In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator

In [2]:
load_dotenv() 
model = ChatGroq(model="openai/gpt-oss-safeguard-20b", temperature=0, max_tokens=1024)

In [3]:
class EvaluationSchema(BaseModel):

    feedback: str = Field(description='Detailed feedback for the essay')
    score: int = Field(description='Score out of 10', ge=0, le=10)

In [4]:
structured_model = model.with_structured_output(EvaluationSchema)

In [5]:
essay = """ Bangladesh in the Age of AI

As the world enters a transformative era defined by artificial intelligence (AI), Bangladesh stands at a crucial turning point — one where it can harness AI to accelerate its development journey or risk widening the technological gap with advanced economies. The age of AI presents immense opportunities along with significant challenges, and how Bangladesh adapts to this transformation will shape its economic progress, governance, and global competitiveness.

Bangladesh’s strengths in the AI domain stem from its rapidly growing ICT sector, a young and dynamic population, and an expanding startup ecosystem. With thousands of engineering and IT graduates entering the workforce each year, the country has the human capital necessary to participate in AI-driven innovation. Universities and research institutions are gradually integrating AI and data science into their curricula, while tech companies and startups are exploring AI solutions in fintech, e-commerce, and digital services. The government’s “Digital Bangladesh” vision and Smart Bangladesh initiative aim to integrate emerging technologies, including AI, into national development strategies.

One of the most promising applications of AI in Bangladesh lies in agriculture, a sector that employs a large portion of the population. AI-powered predictive tools can help farmers optimize crop yields, forecast weather patterns, and manage pest control more efficiently. In healthcare, AI-based diagnostic tools and telemedicine platforms can improve medical access, particularly in rural and underserved areas. Educational platforms are beginning to use AI to personalize learning experiences, while AI-driven governance tools can enhance transparency, service delivery, and fraud detection in public administration.

However, the path toward AI-driven growth is not without obstacles. The digital divide remains a major concern, as rural communities often lack reliable internet access and digital literacy. Automation also poses risks to low-skilled workers in industries such as manufacturing and garments, which are key pillars of the Bangladeshi economy. Without effective reskilling programs and inclusive policies, AI could deepen socio-economic inequalities.

Data privacy and ethical concerns are also emerging challenges. As AI systems depend heavily on data collection and analysis, establishing strong data protection frameworks is essential. Bangladesh must ensure that AI technologies are deployed responsibly, avoiding bias, misuse, and threats to individual privacy.

To responsibly harness AI, Bangladesh must adopt a collaborative approach involving government agencies, academia, industry leaders, and civil society. Policies should encourage innovation while maintaining ethical safeguards. Investment in digital infrastructure, research funding, and skill development will be critical. Additionally, international cooperation and knowledge exchange can help Bangladesh stay aligned with global AI standards and advancements.

Bangladesh’s youthful demographic profile, combined with strategic AI adoption, has the potential to unlock new economic opportunities, strengthen governance, and uplift marginalized communities. However, this potential will only be realized if AI is embraced as a tool for inclusive and human-centered development.

In conclusion, Bangladesh in the age of AI represents a journey of opportunity, responsibility, and transformation. The choices made today will determine whether the nation emerges as a competitive digital economy or struggles to keep pace with global technological change. With careful planning and inclusive policies, AI can become a powerful catalyst for Bangladesh’s sustainable and equitable future. """

In [6]:
prompt = f"""
You are an expert essay evaluator.

Evaluate the language quality of the following essay.
Focus on:
- Grammar
- Vocabulary
- Clarity
- Coherence
- Academic tone

Provide:
1. Detailed feedback
2. A score out of 10 (based only on language quality)

Essay:
{essay}
"""

In [8]:
structured_model.invoke(prompt)

EvaluationSchema(feedback='The essay demonstrates a solid command of academic English, with clear, well‑structured paragraphs and a formal tone appropriate for a policy analysis. Grammar is largely correct; the sentences are varied and mostly free of errors. Minor issues include occasional overuse of the word “AI” without a preceding definition, a few awkward phrasings such as “the age of AI presents immense opportunities along with significant challenges” (could be smoother), and a slight inconsistency in tense when shifting from present to future (“will shape its economic progress” vs. “can help farmers optimize”). Vocabulary is generally precise and context‑appropriate, though some terms could be more sophisticated (e.g., “digital divide” could be expanded to “digital inequality”). The essay is coherent, with logical progression from strengths to challenges to recommendations. The academic tone is maintained throughout, with citations implied but not explicitly referenced. Overall, 

In [9]:
class ESSAYState(TypedDict):

    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float